## Bibliotecas a Utilizar

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Enlaces

In [24]:
enlace_2018_06 = './data/2018_06_Indicadores_Gestion.xls'
enlace_2018_03 = './data/2018_03_Indicadores_Gestion.xls'

enlace_2017_12 = './data/2017_12_Indicadores_Gestion.xls'
enlace_2017_09 = './data/2017_09_Indicadores_Gestion.xls'
enlace_2017_06 = './data/2017_06_Indicadores_Gestion.xls'
enlace_2017_03 = './data/2017_06_Indicadores_Gestion.xls'

enlace_2016_12 = './data/2016_12_Indicadores_Gestion.xls'
enlace_2016_09 = './data/2016_09_Indicadores_Gestion.xls'
enlace_2016_06 = './data/2016_06_Indicadores_Gestion.xls'
enlace_2016_03 = './data/2016_03_Indicadores_Gestion.xls'

enlace_2015_12 = './data/2015_12_Indicadores_Gestion.xls'
enlace_2015_09 = './data/2015_09_Indicadores_Gestion.xls'
enlace_2015_06 = './data/2015_06_Indicadores_Gestion.xls'
enlace_2015_03 = './data/2015_03_Indicadores_Gestion.xls'

enlace_2014_12 = './data/2014_12_Indicadores_Gestion.xls'
enlace_2014_09 = './data/2014_09_Indicadores_Gestion.xls'
enlace_2014_06 = './data/2014_06_Indicadores_Gestion.xls'
enlace_2014_03 = './data/2014_03_Indicadores_Gestion.xls'

enlace_2013_12 = './data/2013_12_Indicadores_Gestion.xls'
enlace_2013_09 = './data/2013_09_Indicadores_Gestion.xls'
enlace_2013_06 = './data/2013_06_Indicadores_Gestion.xls'
enlace_2013_03 = './data/2013_03_Indicadores_Gestion.xls'

enlace_2012_12 = './data/2012_12_Indicadores_Gestion.xls'
enlace_2012_09 = './data/2012_09_Indicadores_Gestion.xls'
enlace_2012_06 = './data/2012_06_Indicadores_Gestion.xls'
enlace_2012_03 = './data/2012_03_Indicadores_Gestion.xls'

## Data Frames

In [25]:
df_2018_06 = pd.read_excel(enlace_2018_06, sheet_name='1 Indicadores Patrimoniales', skiprows=3, header=1)
df_2018_03 = pd.read_excel(enlace_2018_03, sheet_name='1 Indicadores Patrimoniales', skiprows=3, header=1)

df_2017_12 = pd.read_excel(enlace_2017_12, sheet_name='1 Indicadores Patrimoniales', skiprows=3, header=1)
df_2017_09 = pd.read_excel(enlace_2017_09, sheet_name='1 Indicadores Patrimoniales', skiprows=3, header=1)
df_2017_06 = pd.read_excel(enlace_2017_06, sheet_name='1 Indicadores Patrimoniales', skiprows=3, header=1)
df_2017_03 = pd.read_excel(enlace_2017_03, sheet_name='1 Indicadores Patrimoniales', skiprows=3, header=1)

df_2016_12 = pd.read_excel(enlace_2016_12, sheet_name='1 Indicadores Patrimoniales', skiprows=3, header=1)
df_2016_09 = pd.read_excel(enlace_2016_09, sheet_name='1 Indicadores Patrimoniales', skiprows=3, header=1)
df_2016_06 = pd.read_excel(enlace_2016_06, sheet_name='1 Indicadores Patrimoniales', skiprows=3, header=1)
df_2016_03 = pd.read_excel(enlace_2016_03, sheet_name='1 Indicadores Patrimoniales', skiprows=3, header=1)


df_2015_12 = pd.read_excel(enlace_2015_12, sheet_name='1 Indicadores Patrimoniales', skiprows=3, header=1)
df_2015_09 = pd.read_excel(enlace_2015_09, sheet_name='1 Indicadores Patrimoniales', skiprows=3, header=1)
df_2015_06 = pd.read_excel(enlace_2015_06, sheet_name='1 Indicadores Patrimoniales', skiprows=3, header=1)
df_2015_03 = pd.read_excel(enlace_2015_03, sheet_name='1 Indicadores Patrimoniales', skiprows=3, header=1)


df_2014_12 = pd.read_excel(enlace_2014_12, sheet_name='1 Indicadores Patrimoniales', skiprows=3, header=1)
df_2014_09 = pd.read_excel(enlace_2014_09, sheet_name='1 Indicadores Patrimoniales', skiprows=3, header=1)
df_2014_06 = pd.read_excel(enlace_2014_06, sheet_name='1 Indicadores Patrimoniales', skiprows=3, header=1)
df_2014_03 = pd.read_excel(enlace_2014_03, sheet_name='1 Indicadores Patrimoniales', skiprows=3, header=1)


df_2013_12 = pd.read_excel(enlace_2013_12, sheet_name='1 Indicadores Patrimoniales', skiprows=3, header=1)
df_2013_09 = pd.read_excel(enlace_2013_09, sheet_name='1 Indicadores Patrimoniales', skiprows=3, header=1)
df_2013_06 = pd.read_excel(enlace_2013_06, sheet_name='1 Indicadores Patrimoniales', skiprows=3, header=1)
df_2013_03 = pd.read_excel(enlace_2013_03, sheet_name='1 Indicadores Patrimoniales', skiprows=3, header=1)


df_2012_12 = pd.read_excel(enlace_2012_12, sheet_name='1 Indicadores Patrimoniales', skiprows=3, header=1)
df_2012_09 = pd.read_excel(enlace_2012_09, sheet_name='1 Indicadores Patrimoniales', skiprows=3, header=1)
df_2012_06 = pd.read_excel(enlace_2012_06, sheet_name='1 Indicadores Patrimoniales', skiprows=3, header=1)
df_2012_03 = pd.read_excel(enlace_2012_03, sheet_name='1 Indicadores Patrimoniales', skiprows=3, header=1)

## Lista Data Frames

In [26]:
df_list = [df_2018_06, df_2018_03,
           df_2017_12, df_2017_09, df_2017_06, df_2017_03,
           df_2016_12, df_2016_09, df_2016_06, df_2016_03,
           df_2015_12, df_2015_09, df_2015_06, df_2015_03,
           df_2014_12, df_2014_09, df_2014_06, df_2014_03,
           df_2013_12, df_2013_09, df_2013_06, df_2013_03,
           df_2012_12, df_2012_09, df_2012_06, df_2012_03,
          ]

## Trabajando los Data Frames - Función cleanData()
* Creando una columna con fecha para cada Data Frame
* Creando columna con variable de Riesgos de cada dato 
* Reemplazando los datos incorrectos
* Eliminando columnas que no sirven
* Posibilidad de guardar en archivo csv

## Función completa

In [27]:
def cleanData(list_df, ramo):
    
# ================= Variables Limpiar =================    
    # Variable para comletar la fecha de los DataFrames
    fechas = ['06-2018', '03-2018',
            '12-2017', '09-2017', '06-2017', '03-2017',
            '12-2016', '09-2016', '06-2016', '03-2016',
            '12-2015', '09-2015', '06-2015', '03-2015',
            '12-2014', '09-2014', '06-2014', '03-2014',
            '12-2013', '09-2013', '06-2013', '03-2013',
            '12-2012', '09-2012', '06-2012', '03-2012',    
]
    # Variable para completado de columna
    current_cat = 'Patrimoñales y Mixta'
    
    # Variable para el largo de la lista
    largo = len(list_df)
    
# ============== Variables Juntar y Unir ==============
    
    # DF para guardar los df unidos
    df_Unido = pd.DataFrame()
    
    # Variable con nombre de los ramos 
    ramos = ['Patrimoñales y Mixta', 
              'Riesgos del Trabajo', 
              'Transporte Público Pasajeros', 
              'Seguros de Vida', 
              'Seguros de Retiro']
    # Variable para contar las iteraciones
    count = 0
    
    
# ============== Limpieza de Data Frame ===============
    for l, df in zip(range(largo),list_df):

        # Eliminando las primeras 5 filas
        df.drop(df.index[[0,1,2,3,4,5]],inplace=True)

        # Reseteando el Index
        df.reset_index(drop=True, inplace=True)

        # Agregando la columna "Fecha" y "Categoría"
        df['Fecha'] = fechas[l]
        df['Categoría'] = None

        # Completando la columna "Categoría" 
        for i in range(len(df)):
            if pd.isnull(df.iloc[i].NJ):
                current_cat = df.iloc[i]['Denominación']
            else:
                df.loc[i,'Categoría'] = current_cat
        
        # Seteo como categorías 
        df['Categoría'] = df['Categoría'].astype('category', inplace=True)
        
        # Vuelvo a establecesr la primera categoría para la próxima iteración
        current_cat = 'Patrimoñales y Mixta'
        
        # Eliminamos las filas con categoría vacía
        df = df[~df.Categoría.isnull()]

        # Reseteando el Index
        df.reset_index(drop=True, inplace=True)

        # Reemplazando los valores núlos
        df.replace('//',0,inplace=True)
        df.replace('///',0,inplace=True)

        
# ============== Juntando y uniendo Data Frames ===============    

        if count == 0:
            if ramo == 'PAT':
                df_Unido = df_Unido.append(df[ df['Categoría']== ramos[0]], ignore_index=True)
                count += 1
        elif count > 0:
            if ramo == 'PAT':
                df_Unido = df_Unido.append(df[ df['Categoría'] == ramos[0] ], ignore_index=True)
    
    
    df_Unido.to_csv('{}.csv'.format(ramo))
    return 


In [28]:
cleanData(df_list,'PAT')

## Base para limpiar los data frames

In [14]:
def cleanData(list_df, save = False):
    # Variable para comletar la fecha de los DataFrames
    fechas = ['06-2018', '03-2018',
            '12-2017', '09-2017', '06-2017', '03-2017',
            '12-2016', '09-2016', '06-2016', '03-2016',
            '12-2015', '09-2015', '06-2015', '03-2015',
            '12-2014', '09-2014', '06-2014', '03-2014',
            '12-2015', '09-2013', '06-2013', '03-2013',
            '12-2012', '09-2012', '06-2012', '03-2012',    
            ]
    # Variable para completado de columna
    current_cat = 'Patrimoñales y Mixta'
    
    # Variable para el largo de la lista
    largo = len(list_df)
    
    for l, df in zip(range(largo),list_df):

        # Eliminando las primeras 5 filas
        df.drop(df.index[[0,1,2,3,4,5]],inplace=True)

        # Reseteando el Index
        df.reset_index(drop=True, inplace=True)

        # Agregando la columna "Fecha" y "Categoría"
        df['Fecha'] = fechas[l]
        df['Categoría'] = None

        # Completando la columna "Categoría" 
        for i in range(len(df)):
            if pd.isnull(df.iloc[i].NJ):
                current_cat = df.iloc[i]['Denominación']
            else:
                df.loc[i,'Categoría'] = current_cat
        
        # Seteo como categorías 
        df['Categoría'] = df['Categoría'].astype('category', inplace=True)
        
        # Vuelvo a establecesr la primera categoría para la próxima iteración
        current_cat = 'Patrimoñales y Mixta'
        
        # Eliminamos las filas con categoría vacía
        df = df[~df.Categoría.isnull()]

        # Reseteando el Index
        df.reset_index(drop=True, inplace=True)

        # Reemplazando los valores núlos
        df.replace('//',0,inplace=True)
        df.replace('///',0,inplace=True)

        # Salvando el df
        if save == True:
            df.to_csv('df-{}.csv'.format(fechas[l]))

    return 

In [23]:
cleanData([df_2014_03,df_2014_09,df_2016_09], save=True)

/home/ramirogallo/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3798: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


## Base para juntar los data Frames

In [78]:
def juntarUnir(list_df, ramo):
    
    df_Unido = pd.DataFrame()
    
    # Variable con nombre de los ramos 
    ramos = ['Patrimoñales y Mixta', 
              'Riesgos del Trabajo', 
              'Transporte Público Pasajeros', 
              'Seguros de Vida', 
              'Seguros de Retiro']
    # Variable para contar las iteraciones
    count = 0
    
    for df in list_df:
        if count == 0:
            if ramo == 'PAT':
                df_Unido = df_Unido.append(df[ df['Categoría']== ramos[0]], ignore_index=True)
                count += 1
        elif count > 0:
            if ramo == 'PAT':
                df_Unido = df_Unido.append(df[ df['Categoría'] == ramos[0] ], ignore_index=True)
    
    
    df_Unido.to_csv('{}.csv'.format(ramo))
    return df_Unido

In [79]:
#cleanData([df_2018_06, df_2018_03 ],save=True)

#juntarUnir([df_2018_06, df_2018_03 ], ramo='PAT')


#   https://pandas.pydata.org/pandas-docs/stable/merging.html    Para Investigar